In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

In [ ]:
# Set your paths to the folders
image_dir = "C:/Users/aditya/Desktop/project/dogNcat/images"         
annotation_dir = "C:/Users/aditya/Desktop/project/dogNcat/annotations"  

In [ ]:
def convert_label(label):
    return 1 if label.lower() == 'dog' else 0

In [5]:
# Initialize lists to hold image data and labels.
X = []
y = []

# Loop over each XML annotation file.
for xml_file in os.listdir(annotation_dir):
    if not xml_file.endswith('.xml'):
        continue  # Skip any non-XML files

    xml_path = os.path.join(annotation_dir, xml_file)
    
    # Parse the XML file.
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    # Get the image filename from the XML. Adjust the tag if needed.
    filename_tag = root.find('filename')
    if filename_tag is None:
        continue  # Skip if there is no filename tag.
    filename = filename_tag.text
    
    # Get the label from the XML.
    # For example, consider the first object in the annotation.
    object_tag = root.find('object')
    if object_tag is None:
        continue  # Skip if no object is annotated.
    name_tag = object_tag.find('name')
    if name_tag is None:
        continue  # Skip if the object has no name.
    label_str = name_tag.text
    
    # Convert the label to a numerical value.
    label = convert_label(label_str)
    
    # Construct the full path to the image and read it.
    image_path = os.path.join(image_dir, filename)
    img_array = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img_array is None:
        print(f"Warning: Could not read {image_path}")
        continue
    
    # Resize the image to a consistent shape (80x80 here).
    new_img_array = cv2.resize(img_array, (80, 80))
    
    
    X.append(new_img_array)
    y.append(label)

In [6]:
# Convert lists to numpy arrays and reshape images.
X = np.array(X).reshape(-1, 80, 80, 1)
y = np.array(y)

# Normalize the image data.
X = X / 255.0

In [7]:
# Split the dataset into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Build the CNN model.
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(80, 80, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Use 'softmax' if you have more than 2 classes

# Compile the model.
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\aditya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Train the model.
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 11s 101ms/step - accuracy: 0.6532 - loss: 0.6388 - val_accuracy: 0.6585 - val_loss: 0.6075
Epoch 2/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 9s 97ms/step - accuracy: 0.7002 - loss: 0.5665 - val_accuracy: 0.6707 - val_loss: 0.6336
Epoch 3/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 9s 94ms/step - accuracy: 0.7581 - loss: 0.5235 - val_accuracy: 0.7209 - val_loss: 0.5537
Epoch 4/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 9s 101ms/step - accuracy: 0.7994 - loss: 0.4476 - val_accuracy: 0.7073 - val_loss: 0.5926
Epoch 5/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 9s 95ms/step - accuracy: 0.8197 - loss: 0.3970 - val_accuracy: 0.7100 - val_loss: 0.5850
Epoch 6/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 9s 95ms/step - accuracy: 0.8567 - loss: 0.3404 - val_accuracy: 0.7344 - val_loss: 0.6735
Epoch 7/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 9s 97ms/step - accuracy: 0.8817 - loss: 0.2668 - val_accuracy: 0.6653 - val_loss: 0.7613
Epoch 8/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 9s 97ms/step - accuracy: 0.9001 - loss: 0.2378 - val_accuracy: 0.7209 

In [12]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Test Accuracy:", accuracy)


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7165 - loss: 0.8755
Test Accuracy: 0.7100270986557007


In [10]:
# Make predictions on the test set.
predictions = model.predict(X_test)
predicted_val = [int(round(p[0])) for p in predictions]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step


In [11]:
id_line = np.arange(len(predicted_val))  
submission_df = pd.DataFrame({'id': id_line, 'label': predicted_val})
submission_df.to_csv("submission.csv", index=False)